In [2]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import pickle
from utils import flat, trace
from tqdm import tqdm
import psutil
import torch
import gc
from itertools import product
from multiprocessing import Pool
from tqdm import tqdm
from data import RFCXDataset
from model import kaggle_metric
from utils import sigmoid

In [3]:
data_dir = '../../input/kaggle/train'
fnames = os.listdir(data_dir)
recording_ids = [x[:-5] for x in fnames]
print(len(fnames))
train_tp = pd.read_csv('../../input/kaggle/train_tp.csv', index_col='recording_id')
train_fp = pd.read_csv('../../input/kaggle/train_fp.csv', index_col='recording_id')
sample_submission = pd.read_csv('../../input/kaggle/sample_submission.csv')
folds = joblib.load('folds.jl')
print(train_tp.shape)
display(train_tp.head())
print(train_fp.shape)
display(train_fp.head())

4729
(1216, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
003bec244,14,1,44.5440,2531.250,45.1307,5531.25
006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
009b760e6,10,1,50.0854,947.461,52.5293,10852.70


(7781, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
00204008d,21,1,13.8400,3281.2500,14.9333,4125.0000
00204008d,8,1,24.4960,3750.0000,28.6187,5531.2500
00204008d,4,1,15.0027,2343.7500,16.8587,4218.7500
003b04435,22,1,43.2533,10687.5000,44.8587,13687.5000
003b04435,23,1,9.1254,7235.1562,15.2091,11283.3984


In [15]:
sub = pd.read_csv('submission.csv', index_col='recording_id')
display(sub)
display(sub.describe())

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
recording_id,,,,,,,,,,,,,,,,,,,,,,,,
000316da7,0.002449,0.000587,0.000130,0.040953,0.000399,0.003539,0.000166,0.000620,0.000347,0.001547,0.000049,0.000062,0.006835,0.001379,0.000477,0.002109,0.002528,0.000389,0.028606,0.000721,0.001620,0.000957,0.002877,0.003469
003bc2cb2,0.000268,0.001003,0.000153,0.025215,0.000817,0.001826,0.001401,0.000503,0.001812,0.000258,0.000052,0.001100,0.000778,0.001275,0.000676,0.001601,0.543640,0.000394,0.006293,0.000235,0.000343,0.004543,0.001376,0.001749
0061c037e,0.002688,0.000576,0.000789,0.004027,0.003372,0.006573,0.000424,0.001447,0.000156,0.001088,0.320234,0.000887,0.001077,0.000582,0.000157,0.003693,0.011481,0.019750,0.004064,0.001716,0.002058,0.001597,0.000661,0.001174
010eb14d3,0.034840,0.001918,0.000109,0.019581,0.000754,0.000974,0.000328,0.001272,0.169402,0.001078,0.003066,0.000928,0.007772,0.009588,0.001613,0.000790,0.023277,0.001125,0.028263,0.001712,0.001941,0.002318,0.000838,0.000957
011318064,0.000179,0.000682,0.000173,0.012590,0.001443,0.000736,0.000572,0.000144,0.007422,0.000640,0.000192,0.000635,0.002291,0.000581,0.212716,0.017592,0.004490,0.001128,0.029476,0.000688,0.003531,0.005474,0.001546,0.002033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff68f3ac3,0.004087,0.008891,0.001005,0.022530,0.007219,0.152742,0.000122,0.004489,0.001100,0.000102,0.001697,0.000141,0.001102,0.002132,0.000569,0.102751,0.001867,0.006895,0.006123,0.001203,0.000713,0.001252,0.000398,0.432102
ff973e852,0.002006,0.001262,0.000139,0.003940,0.000574,0.000859,0.000590,0.076553,0.000207,0.003844,0.009104,0.000581,0.000545,0.000635,0.000597,0.022215,0.001456,0.011458,0.003011,0.001609,0.001619,0.001503,0.001510,0.006340
ffa5cf6d6,0.000654,0.012839,0.000082,0.014315,0.000210,0.000671,0.000304,0.000461,0.000154,0.002979,0.000419,0.000098,0.000719,0.001659,0.000096,0.141338,0.009502,0.054009,0.000791,0.001417,0.000912,0.003343,0.001900,0.001930


,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
count,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000
mean,0.008219,0.006602,0.012687,0.018004,0.003932,0.016250,0.001627,0.021170,0.008531,0.004742,0.012375,0.011919,0.009573,0.010150,0.011511,0.013059,0.030951,0.009608,0.017175,0.005459,0.003501,0.007096,0.012819,0.029215
std,0.036052,0.018879,0.077865,0.012243,0.027423,0.069205,0.020114,0.046622,0.045789,0.028502,0.080848,0.047294,0.022108,0.058113,0.039052,0.032426,0.109662,0.069406,0.030094,0.055364,0.031583,0.041506,0.072263,0.109783
min,0.000074,0.000064,0.000026,0.000660,0.000106,0.000130,0.000056,0.000077,0.000009,0.000052,0.000024,0.000021,0.000245,0.000323,0.000023,0.000362,0.000319,0.000150,0.000070,0.000080,0.000108,0.000312,0.000262,0.000282
25%,0.000455,0.000456,0.000160,0.009374,0.000461,0.000827,0.000258,0.000589,0.000158,0.000436,0.000154,0.000102,0.000951,0.001315,0.000228,0.001389,0.002536,0.000598,0.001222,0.000520,0.000491,0.001420,0.001187,0.001225
50%,0.000853,0.001171,0.000277,0.015915,0.000788,0.001530,0.000423,0.002386,0.000493,0.000804,0.000365,0.000239,0.001991,0.002215,0.000653,0.002447,0.004901,0.000915,0.005122,0.000832,0.000795,0.002237,0.001694,0.002901
75%,0.002048,0.003683,0.000590,0.023932,0.001499,0.003097,0.000752,0.016392,0.001545,0.001704,0.001186,0.001079,0.005707,0.004643,0.001784,0.006051,0.010796,0.001546,0.022063,0.001469,0.001449,0.003721,0.002619,0.008410
max,0.579189,0.257483,0.766612,0.078675,0.625361,0.685158,0.704359,0.516044,0.618568,0.493694,0.900959,0.524900,0.258747,0.935747,0.368320,0.355169,0.823478,0.959479,0.454189,0.984094,0.815936,0.721421,0.831063,0.887032


In [16]:
sub.mean()

s0     0.008219
s1     0.006602
s2     0.012687
s3     0.018004
s4     0.003932
s5     0.016250
s6     0.001627
s7     0.021170
s8     0.008531
s9     0.004742
s10    0.012375
s11    0.011919
s12    0.009573
s13    0.010150
s14    0.011511
s15    0.013059
s16    0.030951
s17    0.009608
s18    0.017175
s19    0.005459
s20    0.003501
s21    0.007096
s22    0.012819
s23    0.029215
dtype: float64

In [7]:
train_tp

,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
003bec244,14,1,44.5440,2531.250,45.1307,5531.25
006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
009b760e6,10,1,50.0854,947.461,52.5293,10852.70
...,...,...,...,...,...,...
fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75
fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25
ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00
